# 1. Data Preparation

This notebook handles two datasets with Trentino's schools data. In particular, two sources have been found: vivoscuola website and aprilascuola project, which offers a hidden API where it is possible to scrape schools details. 

In [83]:
# Setup
import plotly.graph_objects as go
import textwrap
import json
import pandas as pd
import numpy as np

## Vivoscuola dataset
Starting with vivoscuola dataset, the so called "Istituti Comprensivi" are not taken into consideration, since they are a set of educational units, also called schools. This type of institutions are recognizable through the missing field "Scuola".

In [84]:
# VIVOSCUOLA DATASET
vivo = pd.read_csv("../data/vivoscuola.csv", sep=";")

print("Number of rows: "+str(len(vivo)))
vivo.head()

Number of rows: 997


,Istituto Principale,Scuola,Tipo Istituto,Tipo Gestione,Dirigente,Direttore,Coordinatore Pedagogico,Indirizzo,Comune,Telefono,Fax,Email istituto,Email dirigenza,Email segreteria,Sito web,Codice MIUR
0,ASSOCIAZIONE RUDOLF STEINER PER LA PEDAGOGIA A...,Asilo Colle Fiorito,NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via Stazione Mori, 10 38068",ROVERETO,0464 434899,NaN,NaN,NaN,NaN,NaN,NaN
1,"Scuola dell'infanzia di Breguzzo - Bondo ""Giu...","Scuola dell'Infanzia di Breguzzo - Bondo ""Giu...",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via Marconi - Breguzzo, 5 38087",SELLA GIUDICARIE,0465-901550,NaN,breguzzo.bondo.materna@pec.associazionecoesi.com,NaN,NaN,NaN,NaN
2,"Scuola dell'infanzia di Riva ""Giardino""","Scuola dell'Infanzia di Riva ""Giardino""",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Viale Roma, 32 38066",RIVA DEL GARDA,0464-552360,0464-560456,rivagiardino.materna@pec.associazionecoesi.com,NaN,NaN,NaN,NaN
3,"Scuola dell'infanzia di Lavis ""Madre Maddalen...","Scuola dell'Infanzia di Lavis ""Madre Maddalen...",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via Dei Colli, 4 38015",LAVIS,0461 240366,0461-249078,lavis.materna@pec.associazionecoesi.com,NaN,NaN,NaN,NaN
4,"Scuola dell'infanzia di Rovereto ""Clementino ...","Scuola dell'Infanzia di Rovereto ""Clementino ...",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via S Maria, 58 38068",ROVERETO,0464-420061,NaN,asilovannetti@pec.it,NaN,asilovannetti@gmail.com,NaN,NaN


In [85]:
# Remove Istituti Comprensivi
vivo = vivo[~vivo['Scuola'].isnull()]
print("Number of rows: "+str(len(vivo)))

Number of rows: 724


After removing the institutions that include schools, we are left with 724 school units.

Since there is no column that suggests the grade of the school, the following function tries to detect it from the school name itself. Notice that vivoscuola, compared to aprilascuola, contains schools of all grades, starting from kindergarden to professional training. 

In [86]:

# Infer school type by the complete name
def insert_school_type(df, name_index, ist_index):
    type = []
    for i in range(len(df)):
        s = df.iloc[i, name_index].lower()
        t = ""
        if ("asilo" in s) or ("scuola materna" in s) or ("scuola dell'infanzia" in s):
            t = "Scuola dell'Infanzia"
        elif ("scuola primaria" in s) or ("primaria" in s):
            t = "Scuola Primaria"
        elif ("scuola secondaria di primo " in s) or ("secondaria i" in s):
            t = "Scuola Secondaria di Primo Grado"
        elif ("scuola secondaria di secondo " in s) or ("liceo" in s) or ("istituto tecnico" in s) or ("istituto professionale" in s):
            t = "Scuola Secondaria di Secondo Grado"
        elif ("formazione professionale" in s) or ("formazione professionale" in df.iloc[i, ist_index].lower()):
            t = "Formazione professionale"
        elif "educazione per adulti" in s:
            t = "Educazione per adulti"
        else:
            t = np.nan
        type.append(t)
    df['Tipo Istituto'] = type


insert_school_type(vivo, 1, 0)

We can notice the type of school from the column *"Tipo Istituto"*, inspecting some random rows to get more than one single category of schools. 

In [87]:
vivo.sample(5)

,Istituto Principale,Scuola,Tipo Istituto,Tipo Gestione,Dirigente,Direttore,Coordinatore Pedagogico,Indirizzo,Comune,Telefono,Fax,Email istituto,Email dirigenza,Email segreteria,Sito web,Codice MIUR
571,Scuola dell'infanzia di Madonna di Campiglio,Scuola dell'Infanzia di Madonna di Campiglio,Scuola dell'Infanzia,Paritaria/equiparata,NaN,NaN,ALESSANDRA POMILIO,"Via Serafini - Madonna di Campiglio, 13 38070",TRE VILLE,0465-442779,0465 442779,madonna.campiglio.segretario@fpsm.tn.it,NaN,madonna.campiglio.segretario@fpsm.tn.it,NaN,NaN
523,ISTITUTO COMPRENSIVO CAVALESE,"SCUOLA SECONDARIA DI PRIMO GRADO ""G. SEGANTINI...",Scuola Secondaria di Primo Grado,A carattere statale/provinciale,ROBERTO TROLLI,NaN,NaN,"Piazza Verdi, 6 38033",CAVALESE,0462/340314,0462/342294,ic.cavalese@pec.provincia.tn.it,dir.cavalese@scuole.provincia.tn.it,segr.cavalese@scuole.provincia.tn.it,www.iccavalese.it,TNMM809016
728,"LICEO ""A. Rosmini"" - TRENTO",LICEO DELLE SCIENZE UMANE OPZIONE ECONOMICO SO...,Scuola Secondaria di Secondo Grado,A carattere statale/provinciale,STEFANO KIRCHNER,NaN,NaN,"Via Malfatti, 2 38122",TRENTO,0461/914400,0461/914498,rosmini.trento@pec.provincia.tn.it,dir.rosmini.tn@scuole.provincia.tn.it,segr.rosmini.tn@scuole.provincia.tn.it,www.rosmini.eu,TNPM01000X
275,Scuola dell'infanzia di Campitello di Fassa,Scuola dell'Infanzia di Campitello di Fassa,Scuola dell'Infanzia,Paritaria/equiparata,NaN,NaN,TIZIANA CEOL,"Strèda sènc Felip e Giacum, 9 38031",CAMPITELLO DI FASSA - CIAMPEDEL,0462-750470,NaN,campitello.presidente@fpsm.tn.it,NaN,campitello.segretario@fpsm.tn.it,NaN,NaN
156,ISTITUTO COMPRENSIVO ALTOPIANO DI PINE',"SCUOLA PRIMARIA ""G. DALLAFIOR"" BASELGA DI PINE'",Scuola Primaria,A carattere statale/provinciale,NORMA BORGOGNO,NaN,NaN,"Via delle Scuole, 15 38042",BASELGA DI PINE',0461/558317,0461/558163,ic.pine@pec.provincia.tn.it,dir.ic.altopianodipine@scuole.provincia.tn.it,segr.ic.altopianodipine@scuole.provincia.tn.it,www.icpine.eu,TNEE81601A


Moreover, the column *"Tipo Gestione"* offers a quick view over public (i.e. "Statale") or private schools, also known as charter schools (i.e. "Paritaria"). 

In [88]:
# Map Public and Private school into short version
type = []
for x in vivo['Tipo Gestione']:
    if "Paritaria" in x:
        type.append("Paritaria")
    else:
        type.append("Statale")

vivo['Tipo Gestione'] = type

We can now reorder the columns and remove useless ones. In addition, some columns are renamed, since they will play a mayor role in further code and by being used so frequently, the name should be shorten. 

In [89]:
# Select the columns
vivo = vivo[['Istituto Principale', 'Scuola', 'Tipo Istituto',
             'Tipo Gestione', 'Indirizzo', 'Comune', 'Telefono',
             'Fax', 'Email istituto', 'Email segreteria', 'Sito web', 'Codice MIUR']]

vivo.rename(columns={
    'Istituto Principale': 'Istituto',
    'Scuola': 'Nome',
    'Tipo Gestione': 'Gestione'
}, inplace=True)

The following chunk is necessary to extract the code of the municipality (i.e. CAP) and the address without this code, such that geocoding in the next notebook will be easier. 

In [90]:
# Separate CAP and Address
vivo['CAP'] = [x[-5:] for x in vivo['Indirizzo']]
vivo['Indirizzo'] = [x[:-6] for x in vivo['Indirizzo']]

We can save this dataset inside the Trentino directory inside data, so that original data is not replaced. 

In [91]:
# Save vivo dataset
vivo.reset_index(drop=True, inplace=True) # Resetting properly the index from 0 to n-1
vivo.to_csv("../data/trentino/vivoscuole.csv", index=False)

## Aprilascuola dataset

Aside from vivoscuola dataset, there's also aprilascuola, a project born in the Province of Trento to share school information with citizens. This dataset includes school coordinates and the provincial code, useful to request the number of students and classes per school and per grade. However, it does not consider kindergardens, which instead are present inside vivoscuola dataset. 

Aprilascuola dataset is saved in JSON format and then imported as dataframe. Since the function `pd.read_json()` reads every number as numeric, the school code needs to be assigned separately as strings in order to keep the initial zeros. Also, most of columns are renamed. 

In [92]:
# APRILASCUOLA DATASET (no kindergarden)
apri = pd.read_json(
    "https://aprilascuola.provincia.tn.it/sei//api/istituzioneScolastica/istituzioni/ricerca", 
    dtype={'codiceProvinciale':'str', 'idPadre':'int'})
# Rename columns
apri.rename(columns={
    'idPadre': 'Id Istituto',
    'codiceProvinciale': 'Id',
    'codiceMiur': 'Codice MIUR',
    'denominazioneUfficiale': 'Nome',
    'latitudeY': 'lat',
    'longitudeX': 'lon',
    'istituzionePadre': 'Istituto',
    'indirizzo': 'Indirizzo',
    'email': 'Email',
    'telefono': 'Telefono',
    'comune': 'Comune'
}, inplace=True)
apri.drop(['idobj'], axis=1, inplace=True)
apri.head()

,Id Istituto,Id,Codice MIUR,Nome,Indirizzo,Email,Telefono,lat,lon,Istituto,Comune
0,NaN,0220629599,None,Centro Formazione Professionale ENAIP - Cles,Via Mitterer 10,cfp.cles@enaip.tn.it,0463/421362,46.369265,11.031445,"{'idobj': None, 'idPadre': None, 'codiceProvin...",CLES
1,1707469.0,0220629699,TNFP251STA,SETTORE INDUSTRIA E ARTIGIANATO,Via Mitterer 10,None,None,46.369265,11.031445,"{'idobj': 1707469, 'idPadre': None, 'codicePro...",CLES
2,NaN,0221319599,None,Centro Formazione Professionale ENAIP - Ossana,Via Di S Antonio 1,cfp.ossana@enaip.tn.it,0463/751102,46.310258,10.745046,"{'idobj': None, 'idPadre': None, 'codiceProvin...",OSSANA
3,1707627.0,0221319699,TNFP252STA,SETTORE SERVIZI,Via Di S Antonio 1,None,None,46.310258,10.745046,"{'idobj': 1707627, 'idPadre': None, 'codicePro...",OSSANA
4,NaN,0222049599,None,Centro Formazione Professionale ENAIP - Primiero,Via Forno - Transacqua 12,cfp.primiero@enaip.tn.it,0439/762057,46.176075,11.833824,"{'idobj': None, 'idPadre': None, 'codiceProvin...",PRIMIERO SAN MARTINO DI CASTROZZA


The column about *"Istituti Comprensivi"* is called *Istituto* and contains nested information about the main institute that encloses individual schools. The following chunk will extract their provincial code and their name. 

In [93]:
# Getting nested information outside
apri['Id Istituto'] = [x['codiceProvinciale'] for x in apri['Istituto']]
apri['Istituto'] = [x['denominazioneUfficiale'] for x in apri['Istituto']]

Since most of the string columns are in capslock, these columns, both from vivo and aprilascuola dataset, are transformed such that only the first letters of each word will be in uppercase. 

In [94]:
# Remove capslock and convert everything to Title
apri[['Istituto', 'Nome', 'Comune']] = apri[['Istituto', 'Nome', 'Comune']
                                            ].applymap(lambda s: s.title() if s != None else None)
vivo[['Istituto', 'Nome', 'Comune']] = vivo[['Istituto', 'Nome', 'Comune']
                                            ].applymap(lambda s: s.title() if s != None else None)

A curious thing to be noticed is that aprilascuola schools should all be included inside the vivoscuola dataset. This happens for all schools except for those whose names is *"Educazione Libera Per Adulti"*. By inspecting the JSON page of these schools, we may notice that they lead to the same information of training education (i.e. *"Formazione Professionale"*), which are included inside *Educazione Libera Per Adulti* (treated as their main institute). This means that these schools data were counted twice. For this reason, they are excluded from aprilascuola dataset.

Moreover, "Istituti Comprensivi" provincial codes are removed. In the end, the following chunk inserts the type of school based on columns 3 and 9 (i.e. school name and institute name).

In [95]:
# Remove Institutes and Remove "Educazione Libera Per Adulti", since these schools are counted twice
apri = apri[(~apri['Id Istituto'].isna()) & (
    apri['Nome'] != "Educazione Libera Per Adulti")]

insert_school_type(apri, 3, 9)

## Mixing data sources

Now that both datasets are cleaned, we can join them, keeping attention to possible repetitions. First, we can distinguish schools between kindergardens (i.e. *"materne"*) and non kindergardens (i.e. *"non_mat"*). The last category represents common schools between apri and vivo, which are joined based on the school's name and affiliated institute. 

In [96]:
# Getting the list of kindergarden and not schools
materne = vivo[vivo['Tipo Istituto'] == "Scuola dell'Infanzia"]
non_mat = vivo[vivo['Tipo Istituto'] != "Scuola dell'Infanzia"]

len(materne)
len(non_mat)

# Merging common schools inside vivo and apri
common = pd.merge(apri, non_mat, how="inner", on=['Nome', 'Istituto'])

In [97]:
common.head()

,Id Istituto,Id,Codice MIUR_x,Nome,Indirizzo_x,Email,Telefono_x,lat,lon,Istituto,...,Gestione,Indirizzo_y,Comune_y,Telefono_y,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR_y,CAP
0,0220629599,0220629699,TNFP251STA,Settore Industria E Artigianato,Via Mitterer 10,None,None,46.369265,11.031445,Centro Formazione Professionale Enaip - Cles,...,Paritaria,"Via Mitterer, 10",Cles,0463/421362,0463/421606,cfp.cles@pec.it,cfp.cles@enaip.tn.it,www.enaiptrentino.it,TNFP251STA,38023
1,0221319599,0221319699,TNFP252STA,Settore Servizi,Via Di S Antonio 1,None,None,46.310258,10.745046,Centro Formazione Professionale Enaip - Ossana,...,Paritaria,"Via Di S Antonio, 1",Ossana,0463/751102,0463/751987,cfp.ossana@pec.it,cfp.ossana@enaip.tn.it,www.enaiptrentino.it,TNFP252STA,38026
2,0222049599,0222049699,TNFP253STA,Settore Industria E Artigianato,Via Forno - Transacqua 12,None,None,46.176075,11.833824,Centro Formazione Professionale Enaip - Primiero,...,Paritaria,"Via Forno - Transacqua, 12",Primiero San Martino Di Castrozza,0439/762057,0439/762833,cfp.primiero@pec.it,cfp.primiero@enaip.tn.it,www.enaiptrentino.it,TNFP253STA,38054
3,0222049599,0222049698,TNFP253STA,Settore Servizi,Via Forno - Transacqua 12,None,None,46.176075,11.833824,Centro Formazione Professionale Enaip - Primiero,...,Paritaria,"Via Forno - Transacqua, 12",Primiero San Martino Di Castrozza,0439/762057,0439/762833,cfp.primiero@pec.it,cfp.primiero@enaip.tn.it,www.enaiptrentino.it,TNFP253STA,38054
4,0221969599,0221969698,TNFP254STA,Settore Industria E Artigianato,Via Caltrezza 13,None,None,46.287118,11.511975,Centro Formazione Professionale Enaip - Tesero,...,Paritaria,"Via Caltrezza, 13",Tesero,0462/813133,0462/813145,cfp.tesero@pec.it,cfp.tesero@enaip.tn.it,www.enaiptrentino.it,TNFP254STA,38038


Let's check whether all Ids inside apri dataset are inside the common dataset created from inner join:

In [98]:
# Check if all apri rows are inside the joined dataframe
for i in apri['Id'].index:
    if apri['Id'][i] not in list(common['Id']):
        print(i)

Since no index is printed, all schools inside apri dataset are inside common dataset. Let's check the vivo schools, excluding kindergardens (i.e. non_mat dataset):

In [99]:
# Check if all vivo rows are inside the joined dataframe
indexes = []
for i in non_mat['Nome'].index:
    if non_mat['Nome'][i] not in list(common['Nome']):
        indexes.append(i)
        print(i)

25
225
318
449
450
679


Some schools are not included inside common. We can check these names with a right join between apri and non_mat datasets based on name and institute of schools. Whenever the Id is missing, the school is present in vivo but not in apri and we can therefore focus on the missing schools of common dataset. 

In [100]:
missing_vivo = pd.merge(apri, non_mat, how="right", on=['Nome', 'Istituto'])
missing_vivo = missing_vivo[missing_vivo['Id'].isna()]

As can be noticed, there are 8 schools that are not inside common dataset, whose information is only detained inside vivo dataset:

In [101]:
missing_vivo

,Id Istituto,Id,Codice MIUR_x,Nome,Indirizzo_x,Email,Telefono_x,lat,lon,Istituto,...,Gestione,Indirizzo_y,Comune_y,Telefono_y,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR_y,CAP
4,NaN,NaN,NaN,Scuola Primaria Pellizzano,NaN,NaN,NaN,NaN,NaN,Istituto Comprensivo Alta Val Di Sole,...,Statale,"Via Dei Baschenis, 7",Pellizzano,0463/751018,0463/750043,ic.altavaldisole@pec.provincia.tn.it,segr.icaltavaldisole@scuole.provincia.tn.it,www.icaltavaldisole.it,TNEE83205C,38020
98,NaN,NaN,NaN,Scuola Primaria Ospedale Infantile Trento,NaN,NaN,NaN,NaN,NaN,Istituto Comprensivo Trento 4,...,Statale,"Largo Medaglie D Oro, 9",Trento,0461/920316,0461/922030,ic.trento4@pec.provincia.tn.it,segr.ictn4@scuole.provincia.tn.it,www.ictrento4.it,TNEE85703P,38122
130,NaN,NaN,NaN,Istituto Tecnico Per Il Settore Economico,NaN,NaN,NaN,NaN,NaN,Istituto Sacro Cuore - Trento,...,Paritaria,"Piazza S Teresa Verzeri, 4",Trento,0461/981465,0461/982734,NaN,istitutocomprensivo@istitutosacrocuore.it,www.istitutosacrocuore.it,TNTD035003,38122
173,NaN,NaN,NaN,Scuola Primaria Casa Serena,NaN,NaN,NaN,NaN,NaN,"Istituto Comprensivo Trento 2 ""J. A. Comenius""",...,Statale,"Via Alle Campanelle, 23",Trento,0461/982113,0461/237554,ic.comenius@pec.provincia.tn.it,segr.ictn2@scuole.provincia.tn.it,www.icomenius.it,TNEE820023,38121
276,NaN,NaN,NaN,Scuola Secondaria Di Primo Grado D'Arte Trento,NaN,NaN,NaN,NaN,NaN,Istituto Comprensivo Trento 4,...,Statale,"Viale Degli Olmi, 26",Trento,0461/920316,0461/922030,ic.trento4@pec.provincia.tn.it,segr.ictn4@scuole.provincia.tn.it,www.ictrento4.it,TNMM210004,38123
277,NaN,NaN,NaN,"Scuola Primaria ""Beato S. Bellesini"" Trento",NaN,NaN,NaN,NaN,NaN,Istituto Comprensivo Trento 6,...,Statale,"Via Stoppani, 3",Trento,0461/236214,0461/230583,ic.trento6@pec.provincia.tn.it,segr.ic.tn6@scuole.provincia.tn.it,www.ictrento6.it,TNEE85801C,38121
312,NaN,NaN,NaN,Settore Industria E Artigianato Serale,NaN,NaN,NaN,NaN,NaN,Istituto Formazione Professionale Servizi Alla...,...,Statale,"Via Asiago, 14",Trento,0461/933147,0461/931682,ifp.trento@pec.provincia.tn.it,ifp.trento@scuole.provincia.tn.it,https://ifpsandropertinitrento.it,NaN,38123
428,NaN,NaN,NaN,Scuola Primaria Vigo Cavedine,NaN,NaN,NaN,NaN,NaN,Istituto Comprensivo Valle Dei Laghi - Dro,...,Statale,"Via Don Negri, 21",Cavedine,0461/864026,0461/340648,ic.valledeilaghi@pec.provincia.tn.it,segr.ic.valledeilaghi@scuole.provincia.tn.it,www.icvalledeilaghidro.it,TNEE84407R,38073


Now we can concatenate the inner join dataset with the right join on non_mat dataset to get all the not kindergarden schools, but first let's rename and drop some columns. Notice that since the only rows where MIUR code does not coincide both on x and y version are those with missing values: 

In [102]:
common[common['Codice MIUR_x'] != common['Codice MIUR_y']]


,Id Istituto,Id,Codice MIUR_x,Nome,Indirizzo_x,Email,Telefono_x,lat,lon,Istituto,...,Gestione,Indirizzo_y,Comune_y,Telefono_y,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR_y,CAP
9,0222059597,0222059692,None,Settore Industria E Artigianato Serale,Via Asiago 14,None,None,46.045498,11.137545,Centro Formazione Professionale Enaip - Villaz...,...,Paritaria,"Via Asiago, 14",Trento,0461/920386,0461/914935,cfp.villazzano@pec.it,cfp.villazzano@enaip.tn.it,www.enaiptrentino.it,NaN,38123
18,0221619597,0221619685,None,Settore Servizi Serale,Viale Dei Colli 17,None,None,45.890608,11.047722,Istituto Formazione Professionale Alberghiero ...,...,Statale,"Viale Dei Colli, 17",Rovereto,0464/439164,0464/435851,ifpa.rovereto@pec.provincia.tn.it,segr.ifpa.rovereto@scuole.provincia.tn.it,www.alberghierorovereto.it,NaN,38068
20,0222059595,0222059670,None,Settore Servizi Serale,Viale Verona 141,None,None,46.052671,11.131164,Istituto Formazione Professionale Servizi Alla...,...,Statale,"Viale Verona, 141",Trento,0461/933147,0461/931682,ifp.trento@pec.provincia.tn.it,ifp.trento@scuole.provincia.tn.it,https://ifpsandropertinitrento.it,NaN,38123
178,0222059599,0222059694,None,Settore Servizi Serale,Via Borsieri 2,None,None,NaN,NaN,Centro Formazione Professionale Upt - Trento,...,Paritaria,"Via Borsieri, 2",Trento,0461/239997,0461/260235,cfp-upt@pec.it,segreteria.sede@cfp-upt.it,www.cfp-upt.it,NaN,38122
182,0221619599,0221619695,None,Settore Industria E Artigianato Serale,Piazzale Orsi 1,None,None,45.891881,11.034906,Giuseppe Veronesi - Centro Di Istruzione Scola...,...,Paritaria,"Piazzale Orsi, 1",Rovereto,0464/433484,0464/436873,iscrizioni.veronesi@pec.it,segreteria.dir@cfpgveronesi.it,www.cfpgveronesi.it,NaN,38068
448,0221049598,2013,None,Settore Servizi Serale,Via Ziehl 5,None,None,NaN,NaN,Istituto Formazione Professionale Alberghiero ...,...,Statale,"Via Ziehl, 5",Levico Terme,0461/706294,0461/702025,ifpa.levicoterme@pec.provincia.tn.it,segr.ifpa.levicoterme@scuole.provincia.tn.it,www.alberghierolevico.it,NaN,38056


Since vivoscuola addresses and contact information are well formatted and more complete than aprilascuola's, we will keep them and discard their "y" version belonging to apri dataset:

In [103]:
# Keep vivoscuola addresses and contact information, since they are well formatted and complete
common.rename(columns={
    'Indirizzo_y': 'Indirizzo',
    'Telefono_y': 'Telefono',
    'Comune_y': 'Comune',
    'Tipo Istituto_y': 'Tipo Istituto',
    'Codice MIUR_y': 'Codice MIUR'}, inplace=True)
common.drop(['Codice MIUR_x', 'Indirizzo_x', 'Telefono_x', 'Email',
             'Comune_x', 'Tipo Istituto_x', ], axis=1, inplace=True)

Now we can concatenate common dataset with not-kindergarden's whose name, address and institute match with those inside missing_vivo dataset.

In [104]:
# 2. Add the remaining missing schools in Vivo
common = pd.concat([common,
                    non_mat[(non_mat['Nome'].isin(missing_vivo['Nome'])) &
                            (non_mat['Indirizzo'].isin(missing_vivo['Indirizzo_y'])) &
                            (non_mat['Istituto'].isin(missing_vivo['Istituto']))]], axis=0, ignore_index=True)

By re-executing the code forehead mentioned, we can recheck whether all non kindergarden schools inside vivo are present in common dataset. Since the following chunk does not print anything, we can state that all schools are inside common:

In [105]:
# Check if all vivo rows are inside the joined dataframe
indexes = []
for i in non_mat['Nome'].index:
    if non_mat['Nome'][i] not in list(common['Nome']):
        indexes.append(i)
        print(i)

We have collected information about primary, middle and high schools, plus training education schools. We can concatenate these schools with kindergardens saved inside materne dataset to get the complete dataset of all schools in Trentino. 

In [106]:
# Adding scuole materne to the dataset, inserting None where information is missing
common = pd.concat([common, materne], axis=0, ignore_index=True)

Navigating across vivoscuola, an error can be noticed for a **primary school in Cavedine**. It seems like there are two of them (the other is called *Scuola Primaria Vigo Cavedine*), with same data. Browsing inside the school's website, we can notice just one primary school in Cavedine. Since most updated information can be retrieved with ["Scuola Primaria Cavedine"](https://www.vivoscuola.it/Scuole/ISTITUTO-COMPRENSIVO-VALLE-DEI-LAGHI-DRO2/SCUOLA-PRIMARIA-CAVEDINE/(offset)/scuola/(data)/alunniclassi), the other one is erased.

In [109]:
common.loc[(common['Nome'].str.contains("Cavedine")) & (common['Tipo Istituto']=="Scuola Primaria")]

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP
164,0222152901,0220532101,Scuola Primaria Cavedine,45.978474,10.976432,Istituto Comprensivo Valle Dei Laghi - Dro,Scuola Primaria,Statale,"Via Don Negri, 21",Cavedine,0461/864026,0461/340648,ic.valledeilaghi@pec.provincia.tn.it,segr.ic.valledeilaghi@scuole.provincia.tn.it,www.icvalledeilaghidro.it,TNEE84406Q,38073
456,NaN,NaN,Scuola Primaria Vigo Cavedine,NaN,NaN,Istituto Comprensivo Valle Dei Laghi - Dro,Scuola Primaria,Statale,"Via Don Negri, 21",Cavedine,0461/864026,0461/340648,ic.valledeilaghi@pec.provincia.tn.it,segr.ic.valledeilaghi@scuole.provincia.tn.it,www.icvalledeilaghidro.it,TNEE84407R,38073


In [115]:
# Correcting information about a school
common.drop(common.loc[(common['Nome'] == "Scuola Primaria Vigo Cavedine") 
                       & (common['Id'].isna())].index, 
            inplace=True)

Let's check if there are some other duplicates, based on name, institute, municipality and address:

In [116]:
# Check if there are some duplicates
common[common.duplicated(
    subset=['Nome', 'Istituto', 'Comune', 'Indirizzo'], keep=False)]

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP


In the end, the dataset can be saved. The pickle format was chosen in order to preserve data types, in particular related to the Provincial Code of the school that may be necessary to retrieve additional information in aprilascuola API. 

In [117]:
common.to_pickle("../data/Trentino/scuole.pkl")

In [118]:
# Importing the file
schools = pd.read_pickle("../data/Trentino/scuole.pkl")

## Visualizations

In this short section, we will explore the distribution of schools between public and private, but also in terms of type of school (i.e. primary, kindergarden, middle, high and training schools). Since the plot is used within the website, it is interactive and built with plotly. Two main actions can be performed:

1. Hover: when hovering with the cursor, it is possible to read a description, the count (number of schools of that category) and the percentage with respect to the parent category. In the case of public and private school the parent is the total number of schools in Trentino; whereas, related to the specific type of school, the parent is the type of management of the school (i.e. public or private);
2. Click: When clicking on the specific category, the sunburst will be focused on that specific category. 

In [139]:
# Grouping by private/public and type of school
df = schools.groupby(['Gestione', 'Tipo Istituto']
                     ).size().to_frame('count').reset_index()
df.loc[len(df)] = ['', 'Paritaria', sum(
    df[df['Gestione'] == 'Paritaria']['count'])]
df.loc[len(df)] = ['', 'Statale', sum(
    df[df['Gestione'] == 'Statale']['count'])]
df.index = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 0, 6]
df = df.sort_index()
df['ids'] = df['Gestione']+df['Tipo Istituto']
df['colors'] = ["#DD5840","#F16D55","#F16D55","#F16D55","#F16D55","#F16D55",
                "#5A7FAF", "#7594BD","#7594BD","#7594BD","#7594BD","#7594BD","#7594BD"]


def customwrap(s, width=13):
    return "<br>".join(textwrap.wrap(s, width=width))


df['Tipo Istituto'] = df['Tipo Istituto'].map(customwrap)

In [141]:
# Show proportion of schools per type and management
import plotly.graph_objects as go
fig = go.Figure(go.Sunburst(values=df["count"],
                            ids=df['ids'],
                            labels=df["Tipo Istituto"],
                            parents=df["Gestione"],
                            branchvalues='total',
                            hoverinfo="label+value+percent parent",
                            insidetextorientation='radial',
                            marker=dict(colors=df.colors),
                            maxdepth=2))
fig.update_layout(margin=dict(t=0, l=0, r=0, b=0))
fig.show()
fig.write_html("../viz/sunburst_schools.html")